In [10]:
import pandas as pd

In [11]:
data = pd.read_csv('Standard_Scaler_result.csv',encoding='cp949')

In [12]:

from sklearn.model_selection import train_test_split

In [13]:
features = data[['CBSCORE', 'CREDITOTAMT', 'YSALEAMT',
       'ESTMM',  'IMSAAMT', 'IMJUAMT', 'BUSAAMT', 'BUJUAMT',
       'BU1TOTAMT', 'GAMT', 'LABORCNT', 'KOSPI', '환율', 'GDP', 
       '소상공인체감지수', '실업률', '물가지수', '국고채', '금리', '유가등락률',
       '소비자심리지수','KIND_JOB_A', 'KIND_JOB_B', 'KIND_JOB_C', 'KIND_JOB_D',
       'KIND_JOB_E', 'KIND_JOB_F', 'KIND_JOB_G', 'KIND_JOB_H', 'KIND_JOB_I',
       'KIND_JOB_J', 'KIND_JOB_K', 'KIND_JOB_L', 'KIND_JOB_M', 'KIND_JOB_N',
       'KIND_JOB_O', 'KIND_JOB_P', 'KIND_JOB_Q', 'KIND_JOB_R', 'KIND_JOB_S',
       'KIND_JOB_T']]
act = data['ACTCD']

In [14]:

train_x, test_x,train_y, test_y = train_test_split(features, act,
stratify=act,train_size=0.7,test_size=0.3,random_state=1)

print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(27381, 41) (11736, 41) (27381,) (11736,)


In [15]:
'''
1. ENN
2. ADASYN
3. BORDERLINE SMOTE
4. Tomeklinks
5 SOTEENN
6.SMOTETOMEK
'''
from imblearn.over_sampling import BorderlineSMOTE,ADASYN
from imblearn.combine import SMOTEENN,SMOTETomek
from imblearn.under_sampling import EditedNearestNeighbours, TomekLinks
data_sets={}
enn = EditedNearestNeighbours()
ada = ADASYN(random_state=42)
smte = SMOTEENN(random_state=42)
smtt = SMOTETomek(random_state=42)
tk = TomekLinks()
sm = BorderlineSMOTE(random_state=42)
enn_under_train_x,enn_under_train_y = enn.fit_resample(train_x,train_y)
ada_over_train_x, ada_over_train_y = ada.fit_resample(train_x,train_y)
smte_combine_train_x,smte_combine_train_y = smte.fit_resample(train_x,train_y)
smtt_combine_train_x,smtt_combine_train_y = smtt.fit_resample(train_x,train_y)
tk_under_train_x,tk_under_train_y = tk.fit_resample(train_x,train_y)
sm_over_train_x, sm_over_train_y = sm.fit_resample(train_x,train_y)
data_sets['enn']=[enn_under_train_x,enn_under_train_y]
data_sets['ada']=[ada_over_train_x,ada_over_train_y]
data_sets['smte']=[smte_combine_train_x,smte_combine_train_y]
data_sets['smtt']=[smtt_combine_train_x,smtt_combine_train_y]
data_sets['tk']=[tk_under_train_x,tk_under_train_y]
data_sets['sm']=[sm_over_train_x,sm_over_train_y]
'''
over_train_x : 학습용 독립변수
over_train_y : 학습용 종속변수
'''
keys_list= data_sets.keys()
accuracies = []
matrix_kd_tree_uniform_BOST = []

In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import precision_score,recall_score,f1_score,roc_auc_score,confusion_matrix, classification_report


In [17]:
k = range(1,11)
keys_CM = []

In [18]:
for i in keys_list:
    for j in k : 
        KNC = KNeighborsClassifier(n_neighbors=j)
        KNC.fit(data_sets[i][0],data_sets[i][1])
        pred_y = KNC.predict(test_x)
        #pr = precision_score(y_true=test_y,y_pred=pred_y)
        #r = recall_score(y_true=test_y,y_pred=pred_y)
        #f1 = f1_score(y_true=test_y,y_pred=pred_y)
        #rca = roc_auc_score(y_true=test_y,y_score=pred_y)
        #accuracies.append(KNC.score(test_x, test_y))
        cfm = confusion_matrix(y_true=test_y,y_pred=pred_y)
        #matrix_kd_tree_uniform_BOST.append((j,pr,r,f1,rca,KNC.score(test_x, test_y),cfm))
        keys_CM.append([i,j,cfm,cfm[1][1]/(cfm[0][1]+cfm[1][1])])
    

In [19]:
sorted(keys_CM,key=lambda x: x[3],reverse=True)

[['tk',
  10,
  array([[10603,    63],
         [ 1025,    45]], dtype=int64),
  0.4166666666666667],
 ['tk',
  2,
  array([[10443,   223],
         [  915,   155]], dtype=int64),
  0.41005291005291006],
 ['tk',
  4,
  array([[10473,   193],
         [  958,   112]], dtype=int64),
  0.36721311475409835],
 ['tk',
  6,
  array([[10516,   150],
         [  990,    80]], dtype=int64),
  0.34782608695652173],
 ['tk',
  9,
  array([[10528,   138],
         [  997,    73]], dtype=int64),
  0.3459715639810427],
 ['tk',
  8,
  array([[10562,   104],
         [ 1017,    53]], dtype=int64),
  0.3375796178343949],
 ['tk',
  5,
  array([[10356,   310],
         [  919,   151]], dtype=int64),
  0.3275488069414317],
 ['tk',
  7,
  array([[10454,   212],
         [  967,   103]], dtype=int64),
  0.326984126984127],
 ['enn',
  10,
  array([[10347,   319],
         [  919,   151]], dtype=int64),
  0.32127659574468087],
 ['enn',
  2,
  array([[10068,   598],
         [  802,   268]], dtype=int64),
  0.30